In [1]:
! kaggle

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [2]:
! mkdir /root/.kaggle
! cp kaggle.json /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! chmod 600 /root/.kaggle/kaggle.json

In [4]:
! kaggle datasets download ellenyusa/4classimages

 86% 33.0M/38.5M [00:01<00:00, 9.92MB/s]
100% 38.5M/38.5M [00:01<00:00, 25.0MB/s]


In [ ]:
! unzip -qq 4classimages.zip

In [ ]:
from keras.layers import Dense, Input, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.models import Model

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator, load_img

In [ ]:
img_gen = ImageDataGenerator(rotation_range=10, shear_range=.2, zoom_range=.1)

In [8]:
gen = img_gen.flow_from_directory("data/data")

Found 808 images belonging to 4 classes.


In [ ]:
x_batch, y_batch = gen.next()

In [10]:
x_batch.shape

(32, 256, 256, 3)

In [ ]:
from keras.applications import ResNet50

In [12]:
res = ResNet50(include_top=False, weights='imagenet', input_shape=(256, 256, 3), classes=4)

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 8s 0us/step


In [ ]:
# res.layers

In [ ]:
for layer in res.layers:
  layer.trainable = False

In [15]:
res.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [16]:
flat = Flatten()(res.output)
d1 = Dense(400, activation="tanh")(flat)
d2 = Dense(100, activation="tanh")(d1)
drop = Dropout(rate=0.2)(d2)
d3 = Dense(4, activation="softmax")(drop)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
model = Model(res.input, d3)

model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=["accuracy"])

In [18]:
model.fit_generator(gen, steps_per_epoch=5, epochs=5)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
5/5 [==============================] - 10s 2s/step - loss: 0.8125 - acc: 0.7250
Epoch 2/5
5/5 [==============================] - 1s 250ms/step - loss: 0.3213 - acc: 0.8529
Epoch 3/5
5/5 [==============================] - 1s 175ms/step - loss: 0.2185 - acc: 0.9062
Epoch 4/5
5/5 [==============================] - 2s 389ms/step - loss: 0.1953 - acc: 0.9313
Epoch 5/5
5/5 [==============================] - 2s 473ms/step - loss: 0.1739 - acc: 0.9562


In [26]:
! wget "https://www.petmd.com/sites/default/files/senior-golden-retriever-with-ball-picture-id488657289.jpg"

--2019-11-10 08:46:14--  https://www.petmd.com/sites/default/files/senior-golden-retriever-with-ball-picture-id488657289.jpg
Resolving www.petmd.com (www.petmd.com)... 104.16.83.51, 104.16.84.51
Connecting to www.petmd.com (www.petmd.com)|104.16.83.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83706 (82K) [image/jpeg]
Saving to: ‘senior-golden-retriever-with-ball-picture-id488657289.jpg’

senior-golden-retri 100%[===================>]  81.74K  --.-KB/s    in 0.05s   

2019-11-10 08:46:14 (1.57 MB/s) - ‘senior-golden-retriever-with-ball-picture-id488657289.jpg’ saved [83706/83706]



In [ ]:
img = load_img("senior-golden-retriever-with-ball-picture-id488657289.jpg", target_size=(256, 256))

In [ ]:
np_img = np.array(img)

In [29]:
model.predict(np.array([np_img])).argmax(axis=1)

array([2])

In [30]:
gen.class_indices

{'Humans': 0, 'cats': 1, 'dogs': 2, 'horses': 3}